<a href="https://colab.research.google.com/github/vrushankkk/Image-Compression-using-Huffman-Coding/blob/main/Movie_Recommendation_k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install factor_analyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 762 kB/s 
  Created wheel for factor-analyzer: filename=factor_analyzer-0.4.0-py3-none-any.whl size=41455 sha256=c0e89eb1b890e0387e6ffac1fd5253846667e51870dcc4db339d843477a1437b
  Stored in directory: /root/.cache/pip/wheels/ac/00/37/1f0e8a5039f9e9f207c4405bbce0796f07701eb377bfc6cc76
Successfully built factor-analyzer


In [2]:
from google.colab import files

uploaded = files.upload()


Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv


In [3]:
#All the header files required for the code
import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [4]:
#Importing both the file using pandas 
data1 = pd.read_csv('movies.csv')
data2 = pd.read_csv('ratings.csv')

In [5]:
#Deleting unnecessary columns

data2 = data2.drop('timestamp',axis = 1)

In [6]:
data1

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
data2

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [8]:
#Merging both the dataframes
data = pd.merge(data2 , data1 , how='outer', on='movieId')

In [9]:
data

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100849,NaN,30892,NaN,In the Realms of the Unreal (2004),Animation|Documentary
100850,NaN,32160,NaN,Twentieth Century (1934),Comedy
100851,NaN,32371,NaN,Call Northside 777 (1948),Crime|Drama|Film-Noir
100852,NaN,34482,NaN,"Browning Version, The (1951)",Drama


In [10]:
# Data Processing
# Converting Genres into different columns 

x = data.genres
a = list()
for i in x:
    abc = i
    a.append(abc.split('|'))
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    data[i] = 0
data.head(2)

,userId,movieId,rating,title,genres,Adventure,Comedy,Action,Mystery,Crime,...,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# we assign 1 to all the columns which are present in the Genres
for i in b:
    data.loc[data['genres'].str.contains(i), i] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
data.head(2)

,userId,movieId,rating,title,genres,Adventure,Comedy,Action,Mystery,Crime,...,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
# Now there is no use of genre and movie name
data = data.drop(['genres','title',],axis =1)
data.head()

,userId,movieId,rating,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,...,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,1.0,1,4.0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,5.0,1,4.0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,7.0,1,4.5,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,15.0,1,2.5,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,17.0,1,4.5,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [14]:
data.columns

Index(['userId', 'movieId', 'rating', 'Adventure', 'Comedy', 'Action',
       'Mystery', 'Crime', 'Thriller', 'Drama', 'Animation', 'Children',
       'Horror', 'Documentary', 'Sci-Fi', 'Fantasy', 'Film-Noir', 'Western',
       'Musical', 'Romance', '(no genres listed)', 'War'],
      dtype='object')

In [15]:
# Because of merging some null values are created
data.isnull().sum()

userId                18
movieId                0
rating                18
Adventure              0
Comedy                 0
Action                 0
Mystery                0
Crime                  0
Thriller               0
Drama                  0
Animation              0
Children               0
Horror                 0
Documentary            0
Sci-Fi                 0
Fantasy                0
Film-Noir              0
Western                0
Musical                0
Romance                0
(no genres listed)     0
War                    0
dtype: int64

In [16]:
#WE simply drop the null values coz the are not treatable
data.dropna(inplace= True )

In [17]:
data.isnull().sum()

userId                0
movieId               0
rating                0
Adventure             0
Comedy                0
Action                0
Mystery               0
Crime                 0
Thriller              0
Drama                 0
Animation             0
Children              0
Horror                0
Documentary           0
Sci-Fi                0
Fantasy               0
Film-Noir             0
Western               0
Musical               0
Romance               0
(no genres listed)    0
War                   0
dtype: int64

In [18]:
#creation of clusters 
kmeanModel = KMeans(n_clusters=8)
kmeanModel.fit(data)

KMeans()

In [19]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

80316    0
78790    2
81624    0
25960    0
61640    0
62405    0
9630     0
97595    7
50234    1
64230    0
Name: Cluster, dtype: int32

In [20]:
data['Cluster'].value_counts()

0    77939
1     6396
5     4070
4     3810
2     3278
7     2980
6     1297
3     1066
Name: Cluster, dtype: int64

In [21]:
data.head()

,userId,movieId,rating,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,...,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War,Cluster
0,1.0,1,4.0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,5.0,1,4.0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,7.0,1,4.5,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,15.0,1,2.5,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,17.0,1,4.5,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [22]:
# When we merge the dataframe for a single movie multiple rows were created so a single movie is allotted
# to many clusters so here we allot a single cluster to a movie 
# the Cluster which occurs maximum number of times is alloted to the movie  
e = []
def fi(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index 
    c = [a['movieId'][d[0]],int(b.idxmax())]
    e.append(c)
    

In [23]:
data.groupby('movieId').apply(lambda x: fi(x))

""


In [24]:
e = pd.DataFrame(e)

In [25]:
e.head()

,0,1
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [26]:
#  column name shifts on its own so to debug that
# Here just the column names are swapped
e.rename(columns = {0:'movieId',1:'Cluster'},inplace=True)
e.drop_duplicates(inplace=True)

In [27]:
e.head(10)

,movieId,Cluster
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0


In [28]:
data1 = pd.read_csv('movies.csv')
new_data = pd.merge(e , data1 , how='outer', on='movieId')

In [29]:
# These were the movies we deleted while merging the file  
new_data.isnull().sum()

movieId     0
Cluster    18
title       0
genres      0
dtype: int64

In [30]:
# We can delete the movies but I just label them randomly :)
new_data.fillna(random.randint(0,8),inplace=True)

In [31]:
new_data.isnull().sum()

movieId    0
Cluster    0
title      0
genres     0
dtype: int64

In [32]:
#This function select the cluster for a user according the the user choice
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,3883))
    for i in l:
        print(new_data['movieId'][i] , new_data['title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data['Cluster'][new_data.movieId == l]

In [33]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['title'][new_data.Cluster == int(l)].sample(n=10))
        print('--------------------------------------------------------------------')
        print('Do you like these movies(y/n)')
        abc = input()
        while ((abc =='y') or (abc == 'Y')):          
            print(new_data['title'][new_data.Cluster == int(l)].sample(n=10))
            print('--------------------------------------------------------------------')
            print('Want more!!!!(y/n)')
            abc = input()
            if ((abc =='N') or (abc == 'n')):
                ans =True

In [34]:
main()

Select The Movies Id you would like to watch:
778--->Trainspotting (1996)
1426--->Zeus and Roxanne (1997)
639--->Girl 6 (1996)
4409--->Shadows and Fog (1991)
2307--->One Tough Cop (1998)
3739--->Trouble in Paradise (1932)
2549--->Wing Commander (1999)
3740--->Big Trouble in Little China (1986)
2959--->Fight Club (1999)
3057--->Where's Marlowe? (1998)
2901--->Phantasm (1979)
2748--->Allan Quatermain and the Lost City of Gold (1987)
3434--->Death Wish 5: The Face of Death (1994)
252--->I.Q. (1994)
579--->Escort, The (Scorta, La) (1993)
--------------------------------------------------------------------
778
2265               Last Night (1998)
4492                  Tremors (1990)
4335     Shape of Things, The (2003)
4256     Bend It Like Beckham (2002)
2778                   Hamlet (1990)
3049        Born in East L.A. (1987)
2571         Band of the Hand (1986)
1002             Paris, Texas (1984)
364     Beverly Hills Cop III (1994)
4476        American Splendor (2003)
Name: title, dtyp